# Single EM27/SUN retrieval demo

Before beginning, please follow the installation instructions in the README of the code repository. Julia needs to be installed, and the required packages must be available on the computer that this notebook runs on. Further, the example data must be present (spectroscopy, solar model, pre-processed EM27 measurements with prior atmosphere).

In this demonstration notebook, we will use pre-processed PROFFAST data (spectra, model atmosphere data) to perform a single retrieval using the spectral window of your choosing.

**Important notes before starting!**

Some of the chosen data structures might feel overly complicated and redundant. This is partly due to the fact that the presented retrieval set-up is also capable of doing multi-band retrievals, i.e. retrieve XCO2 using more than one absorption band. While this workbook only uses one spectral window at a time, it would be relatively straightforward do extend the code to perform a multi-band retrieval.

## Initial set up, loading of modules and functions

In [ ]:
# Activate the local project directory. This allows us to have an encapsulated
# area with the needed packages/modules available without running into version
# conflicts with the "general" Julia environment.
using Pkg; Pkg.activate("./");

In [ ]:
# Load various packages/modules that we need to make the code work

using CSV, DataFrames
using Dates
using Glob
using Interpolations
using LinearAlgebra
using Logging, LoggingExtras
using Plots; default(fmt = :svg, bottom_margin = 10Plots.mm, left_margin = 10Plots.mm, top_margin = 10Plots.mm, right_margin = 10Plots.mm)
using Plots.PlotMeasures
using Printf
using ProgressMeter

using Unitful
using Statistics 
using YAML

using RetrievalToolbox; const RE = RetrievalToolbox;
# (ignore warnings about XRTM, we do not currently need the XRTM library to do radiative transfer!)

The following lines load some user-defined functions. These are mostly helper functions that make certain tasks easier, e.g. load PROFFAST files, calculate the instrument response functions, etc.

In [ ]:
# Load code to deal with PROFFAST pre-processing outputs
include("PROFFAST_tools.jl");

# Load code to deal with retrieval preparation
include("EM27-retrieval.jl");

# Load the code that contains the forward model
include("forward_model.jl");

# Load the code that processes a scene
# (set up the state vector contents, creates a solver, iterations until convergence is reached)
include("process_scene.jl");

## Retrieval preparation

In this EM27/SUN retrieval demo, we are storing the configuration of the potential retrieval windows in a YAML file called `windows.yml`. Feel free to open that file and study its contents. Each spectral window is identified by a number which represents the order in which the window (generally?) appears in the PROFFAST "invers input file". Look at the `inp_fast` folder inside PROFFAST (or PROFFASTpylot) for a file named `invers24*.inp`, which may have those windows listed. Each entry also includes information whether the spectral data is found in a "SN" or "SM" spectral file, which denotes whether the meaningful data was recorded with the first channel (SN) or the second channel (SM). Finally, each entry requires spectroscopy data: there we define which gases we want to include in the retrieval, and which spectroscopy file and VMR unit we want to use.

**NOTE: This configuration is not part of the core RetrievalToolbox functionality. It is mostly a convenient way of organizing your inputs. We could have arranged this in many ways, but YAML seems a useful method of doing so.**

### Set configuration

In [ ]:
# Load the YAML configuration file
win_yml = YAML.load_file("windows.yml");

# Get a list of ALL spectral window indices, turn them into Integers, and sort
# (useful to have a list of ALL windows)
all_idx = keys(win_yml) |> collect .|> Int |> sort

#### Intermission: the SM and SN spectra files, and their purpose

The function `read_PROFFAST_spectrum` below grabs a mesurement file `*.BIN` and then converts the contents into an easy-to-use dictionary. Note that the *SN.BIN files contain spectra from the *MIR* (mid infra-red) band, up until ~5300 cm-1, 
whereas the *SM.BIN files contain spectral from the *NIR* (near infra-red) band, from ~5500 cm-1 up to ~12500 cm-1. Thus, if we choose a spectral window at ~4800 cm-1, like the strong CO2 band, then we naturally want to use the measurement recorded at the MIR detector. For a spectral window like the O2 absorption at ~7800 cm-1, we must use the NIR detector.

Below is a helpful plot that shows the spectral coverage of the two detectors. Note that not all EM27/SUN instruments come shipped with the MIR detector.

In [ ]:
sm = read_PROFFAST_spectrum("example_data/spectra/250722_131526SM.BIN");
sn = read_PROFFAST_spectrum("example_data/spectra/250722_131526SN.BIN");

plot(sm["wavenumber"], sm["spectrum"], label="SM")
plot!(sn["wavenumber"], sn["spectrum"], label="SN")
xlabel!("Wavenumber cm-1")
ylabel!("Signal")

In [ ]:
# Observe the contents of the measurement dictionary:
sm

### Load measurements from a folder

Below code snippet will read all measurements from the specified folder and convert them into dictonary-type data. The function `read_measurements` also selects the portions of the spectrum that corresponds to the different spectral windows, so we only keep the measurement data that is relevant.

<div class="alert alert-block alert-info">
<b>USER INPUT:</b> Edit the folder below to point to some location with has some set of measurements of interest!
</div>

In [ ]:
#=
    Grab the folder with our EM27 measurements. Important - this must be the folder which contains the pre-processed spectra!
    The resulting dictionary is now a "collection of collections" that should be used in the following way:

        measurements[1] => this gives us all measurements from the folder

        measurements[1][fname1] => this gives us the measurement `fname1` from the folder,
                                   that has been subselected to only contain the spectrum
                                   relevant to the spectral window with index `1`

        measurements[3][fname1] => this gives us the measurement `fname1` from the folder,
                                   that has been subselected to only contain the spectrum
                                   relevant to the spectral window with index `3`

        Both measurements[1][fname1] and measurements[3][fname1] have the same metadata, (lat, lon, time, ...), but only
        vary in terms of the fields: `spectrum`, `wavenumber`, `start_wavenumber`, `end_wavenumber`

        measurements[5][fname1] => this now has the spectrum/wavenumber data coming from the *SM.BIN data, since
                                   the spectral window with index `5` requires the data to be read from that
                                   source (as given in `windows.yml`.
=#

##################

measurements = read_measurements(
    "./example_data/spectra/", #### ===> Possibly edit this line! <===
    win_yml, # Configuration YAML
    all_idx # Load all spectral windows for the sake of demonstration
);

In [ ]:
#= 
    For some functions later on, it is useful to have a filename at hand. Some of the file contents
    may be the same for every file, so it does not matter which file we read them from.

    Of course, depending on the spectral window choice, this may be a *SN.BIN or a *SM.BIN file.

=#
fname_first = Dict(
    idx => collect(keys(measurements[idx])) |> sort |> first
    for idx in all_idx # Get them for ALL spectral windows
    )

### Visualize the measurements for each spectral range

Below is a series of plots that show the measurements corresponding to the available spectral windows that we set in `windows.yml`. For the sake of illustration the code below just shows the first measurement present in the directory that you chose above. Observe that `measurements` is a dictionary which has keys corresponding to the spectral window numbers from `windows.yml`. Within that is another dictionary which contains the contents of the measurement that can be accessed by the filenames.

In [ ]:
for idx in sort(collect(keys(win_yml)))

    meas = measurements[idx][fname_first[idx]]
    
    p = plot()
    plot!(p, meas["wavenumber"], meas["spectrum"], label=nothing, size=(1000, 400))
    xlabel!("Wavenumber [cm-1]")
    
    title!("Spectral window #$(idx): $(win_yml[idx]["name"])")
    display(p)

end

### Selecion of spectral window to retrieve

<div class="alert alert-block alert-info">
<b>USER INPUT:</b> In the next cell we choose which retrieval window to use. Select one of the numbers from the `windows.yml` file (1-7) and/or consult the plots just above. Feel free to return to this point later on and change the variable. Note that you must re-run <b>all</b> cells from here onward for the changes to take full effect!
</div>

In [ ]:
idx_want::Int = 1

In [ ]:
# Helpful to have all measurement file names for this window choice:
all_fnames = collect(keys((measurements[idx_want]))) |> sort

The next step creates a so-called spectral window according to the specifications that are laid out in the `windows.yml` file. We supply a `buffer` keyword here that makes sure that the high-resolution model grid extends far enough beyond our desired window to allow for the ISRF application at the edge points. We choose this buffer to be some value that should be big enough, in this case ~15 wavenumbers. Look at the ISRF (below) to make sure that the buffer is **larger** than half the width of the ISRF waveform.

In [ ]:
swin_list = create_spectral_windows(
        win_yml,
        idx_want,
        buffer=15.0
    )

Similarly, based on the listed spectroscopy objects, we create a gas object for each gas. **NOTE** that this step also loads the large spectroscopy objects into memory, hence this step may take a while.

In [ ]:
gas_dict = create_gases(
    win_yml, # Windows YAML file that defines our spectral ranges
    20, # Number of pressure levels in model gas
    Float64, # Number type (leave this always as Float64)
    idx_want # List of spectral window identifiers
);

### Create the Instrument Spectral Response Function (ISRF)

We calculate the ISRF "on-the-fly" with code that was copied from PROFFAST. The only parameter needed to calculate the ISRF is the so-called modulation efficiency, which is stored in the PROFFAST-preprocessed spectrum files. The ISRFs are pre-calculated and stored in a table. Every spectral sample thus has its own ISRF. This is necessary since the ISRF changes spectrally and we must model this wavenumber-dependent ISRF. The most convenient way is to pre-calculate the ISRF for each spectral sample and then let the software pick the right one during the ISRF application.

In [ ]:
# Produce ISRFs for each window (window index -> ISRF table)
isrf_dict = Dict()

for idx in keys(measurements)

    # We only need the first measurement, they should all be the same anyway..
    s = first(values(measurements[idx]))
    
    isrf = create_isrf_tables(s["wavenumber"], s["modulation_efficiency"]...)
    isrf_dict[idx] = isrf
    
end

#### Plot the ISRF

In [ ]:
plot_idx = 1 # Plot the ISRF for this spectral position
nu_at_idx = first(values(measurements[idx_want]))["wavenumber"][plot_idx] # Get the wavenumber

p = plot()
      
plot!(
    p,
    isrf_dict[idx_want].ww_delta[:, plot_idx],
    isrf_dict[idx_want].relative_response[:, plot_idx],
    lw=2.0,
    label="ν = $(nu_at_idx) cm-1"
)

xlabel!("Δν [cm\$^{-1}\$]\n")
ylabel!("\nISRF [cm]")

### Create dispersion objects

Dispersion objects map out the relationship between spectral samples and their wavenumbers. It is important for this relationship to be accurate enough that we can match the modeled spectral features to the measurements. Again, like before, we create these objects and move them into a dictionary where can access them later on.

In [ ]:
dispersions = Dict(
    idx => create_dispersion(
            first(values(measurements[idx])),
            swin
    ) for (idx, swin) in swin_list
)

for swin in values(swin_list)
    println("$(swin) has $(swin.N_hires) spectral points")
end

### Interpolate the model atmosphere to the time of measurement

With the PROFFAST workflow, we also get the model atmospheres from the GINPUT tools, derived from GEOS model runs. We take those 3-hourly model files and interpolate the contents to the actual time of measurement. The function that reads those MAP files automatically converts the wet gas mole fractions into dry ones.

In [ ]:
 # Interpolate MAP atmospheres to measurement times   
map_per_file = interpolate_map(
    "./example_data/map/",
    measurements
);

#### Plot the contents of the model atmospheres

In [ ]:
p = plot(size=(450, 550))
for fname in keys(measurements[1])

    map_file = map_per_file[fname]
    
    plot!(
        p,
        map_file["h2o"],
        map_file["Pressure"],
        label=nothing,
        );
end
yflip!()

### Get the pressure data and interpolate at the measurement times

Following line takes all the pressure measurements and supplements the `measurements` dictionaries with the values from the pressure sensor that closest match the measurement time.

In [ ]:
interpolate_pressure_files_B33!(measurements, "./example_data/pressure/b33_20250722.csv")

#### Look at surface pressure data

In [ ]:
_plot_dates = DateTime[]
_plot_pressure = Float64[]
for (_, meas) in measurements[idx_want]
    push!(_plot_dates, meas["datetime"])
    push!(_plot_pressure, meas["pressure"] |> ustrip)
end

scatter(_plot_dates, _plot_pressure, label=nothing, markershape=:circle, size=(1000, 400));
ylabel!("Surface pressure")
xlabel!("Time of measurement UTC")

### Create the state vector!

We must know in advance which state vector elements we plan to use in our retrieval problem. This information is needed to allocate containers that are subsequently used to store Jacobians and other information. For this example, a helper function `create_statevector` (created specficially for this EM27/SUN application), produces the "right" state vector for us. The state vector elements may have no meaningful value in them for the time being, as they will be changed at a later stage. For example, the dispersion polynomial coefficients will be derived from the data inside the `measurements` dictionary, and they will be inserted into the state vector elements before the retrieval begins.

In [ ]:
sv_dict = Dict(idx =>
    create_statevector(
        Dict(dispersions[idx].spectral_window => dispersions[idx]), # A dictionary that maps spectral window => dictonary
        gas_dict[idx], # A vector of gas objects
        1, # Order of solar continuum polynomial
        1, # Order of dispersion polynomial
    ) 
    
    for idx in keys(swin_list)
)

## Scene set-up

RetrievalToolbox users so-called buffers to store intermediate results, as opposed to creating new vectors and arrays at each evaluation step. In order to do that, we must provide two numbers: one that is slightly larger than the number of high-resolution model wavenumbers (`N1`), and one that is slightly larger than the number of detector elements that we use to compare against the models (`N2`). The code below estimates those numbers for a single-band retrieval set-up.

In [ ]:
N1 = swin_list[idx_want].N_hires * 1.1 |> ceil |> Int
N2 = measurements[idx_want][fname_first[idx_want]]["N_wavenumber"] * 1.1 |> ceil |> Int

# Create the `buffer` object!
buf = create_buffers(
    Float64, # Number type to be used for the vectors and arrays
    Dict(d.spectral_window => d for (k,d) in dispersions), # Dictionary [spectral window => dispersion]
    gas_dict[idx_want], # List of gases
    sv_dict[idx_want], # State vector(s)
    N1,
    N2;
    N_RT_lev=20 # 20 levels for our RT calculations!
);

The buffer object `buf` is a big, pre-allocated container that stores almost all needed information to run retrievals. However, the function call above only creates suitable and often empy containers, that must still be filled with meaningful information. For example, the buffer has an empty location:

In [ ]:
buf.scene.location

We add meaningful data by creating our own `EarthLocation` object using the coordinates from the measurement data, and then add it to the buffer object (buffer -> scene -> location). We also make sure that the system understands that we are doing retrievals for an uplooking observer.

In [ ]:
loc = RE.EarthLocation(
    measurements[idx_want][fname_first[idx_want]]["longitude"],
    measurements[idx_want][fname_first[idx_want]]["latitude"],
    measurements[idx_want][fname_first[idx_want]]["altitude"],
    u"km"
    );

buf.scene.location = loc;
buf.scene.observer = RE.UplookingGroundObserver();

# Show the new location object with proper values
buf.scene.location

## Run the retrieval and inspect the result

The code cell below now actually performs one single retrieval for a measurement of choice. The top-level flow is the following: we first pick a number, corresponding to the number of the measurement. We then grab the filename from the list we created earlier (`all_fnames`), and also the measurement dictionary that corresponds to that filename. That measurement is now assigned to the variable `meas`. `meas` contains all the needed per-scene information that changes between measurements (solar angle, the spectrum itself etc.).

We first need to extract the dispersion coefficients from the measurement dictionary: since our dispersion relation is expressed as a polynomial, we can express the first coefficient (order 0, `d0`) as the starting wavenumber and the second coefficient (or the spacing between wavenumbers, order 1, `d1`), as that given by the data as well (stored in `delta_wavenumber`). Note that retrievals for high-resolution spectroscopic measurements are *very sensitive* to this dispersion relationship, and a bad first guess can cause the retrieval to fail (it is not able to "lock in" to the correct value due to the non-linearity of that state vector element).

The `process_spectrum` function takes care of the main process, we just have to pass on the correct values as indicated by the in-line comments in the code cell below.

Once done, we print out some diagnostics, such as the $\chi^2$ statistic of the spectral residual (goodness of fit), as well as the pressure-weighted total columns.

In [ ]:
#include("forward_model.jl")
#include("process_scene.jl")

file_idx = 6 # Pick a number between 1 and length(all_fnames)

fname = all_fnames[file_idx] # File name corresponding to `file_idx`
meas = measurements[idx_want][fname] # Measurement data dictionary corresponding to `fname`

# Dispersion coefficients derived from the measurement
d0 = meas["start_wavenumber"]
d1 = meas["delta_wavenumber"]

# PROCESS!
@time s = process_spectrum(
    Dict(dispersions[idx_want] => meas["spectrum"]), # Meausred spectrum as dictonary: dispersion => spectrum
    meas["datetime"], # Datetime of measurement
    90.0 - meas["elevation_angle"], # Solar zenith angle
    meas["pressure"], # Measured surface pressure (with physical unit!)
    [ # Dispersion coefficients initial
        d0 - d1,
        d1,
        0.0
    ],
    Dict(dispersions[idx_want].spectral_window => dispersions[idx_want]), # Spectral window => dispersion
    meas["noise_std"], # Estimated noise, expressed as the standard deviation of a Gaussian in signal units
    map_per_file[fname], # Model atmosphere prior for this measurement
    sv_dict[idx_want], # The state vector object
    Dict(dispersions[i] => isrf_dict[i] for i in keys(swin_list)), # Dispersion => instrument spectral response function
    buf; # The buffer object
    max_iter=20 # Number of iterations before we give up
);

println("Finished in $(RE.get_iteration_count(s)) iterations")

println("Per-band χ2: ")
for (k,v) in RE.calculate_chi2(s)
    println("    $(k.window_name): $(v)")
end

println("XGAS: ")
for (k,v) in RE.calculate_xgas(buf.scene.atmosphere)
    println("    X$(k): $(v)")
end

In [ ]:
# Print some info about the posterior state
RE.print_posterior(s)

Code below plots the fit over the measurement, which is always a good way to check if the retrival set-up is able to produce a good fit of the observations. Looking at the spectral residuals too can help diagnose problems.

In [ ]:
swin = swin_list[idx_want]

p = plot(layout=(2, 1), size=(1000, 650))
snr_label = @sprintf "SNR: %.0f" ((RE.get_measured(s) ./ RE.get_noise(s)) |> mean)


y = RE.get_measured(s, swin);
y2 = RE.get_modeled(s, swin);
eps = RE.get_noise(s, swin);

ww = RE.get_wavenumber(s, swin);

#plot!(twinx(), swin.ww_grid, buf.rt[swin].hires_solar, label="Solar lines"); # plot solar lines?

plot!(ww, y, label="EM27/SUN", linewidth=2.0) #, ylims=(0, 9))
plot!(ww, y2, label="Fit", linewidth=1.0, linestyle=:dash, linecolor="black")
plot!(ww, y2 .- y, label="Residual", linewidth=1.0, linecolor="black")

title!(snr_label, subplot=1)
ylabel!("Signal", subplot=1)
xlabel!("Wavenumber [cm\$^{-1}\$]", subplot=2, font=(10, "FreeSerif"))

rrms = sqrt(mean((100.0 * (y2 .- y) ./ y).^2))

plot!(ww, (y2 .- y) ./ eps, label="Residual", subplot=2)
title!(@sprintf("RRMS: %.4f %%", rrms), subplot=2, titlefont=(10))
ylabel!("Fractions of noise", subplot=2)



xlabel!("Wavenumber [cm\$^{-1}\$]", subplot=2, font=(10, "FreeSerif"))
display(p)

## Run the retrieval in a step-by-step fashion

The function `process_scene` is very helpful when wanting to run the full retrieval until it either converges or runs out of iterations. If we want to slowly inspect what happens step by step, we have to make slight changes to the set up. First, we do mostly the same as before, but we set the number of maximal iterations to `0`. The function returns a "solver object", which is the central object that the inversion functions that ship with RetrievalToolbox operate on.

In [ ]:
RE.reset!(s.state_vector)

s = process_spectrum(
    Dict(dispersions[idx_want] => meas["spectrum"]), # Meausred spectrum as dictonary: dispersion => spectrum
    meas["datetime"], # Datetime of measurement
    90.0 - meas["elevation_angle"], # Solar zenith angle
    meas["pressure"], # Measured surface pressure (with physical unit!)
    [ # Dispersion coefficients initial
        d0 - d1,
        d1,
        0.0
    ],
    Dict(dispersions[idx_want].spectral_window => dispersions[idx_want]), # Spectral window => dispersion
    meas["noise_std"], # Estimated noise, expressed as the standard deviation of a Gaussian in signal units
    map_per_file[fname], # Model atmosphere prior for this measurement
    sv_dict[idx_want], # The state vector object
    Dict(dispersions[i] => isrf_dict[i] for i in keys(swin_list)), # Dispersion => instrument spectral response function
    buf; # The buffer object
    max_iter=0 # Number of iterations before we give up
);

If we now want to observe the fit progress step by step, we can manually call for the solver object `s` to perform another iteration. An iteration evaluates the forward model **and** the Jacobians, the inversion algebra then adjusts the state vector contents to minimize the cost function (the difference to the measurement). Call the cell below repeatedly to observe how the retrieval (hopefully) moves its model prediction closer to the observation!

In [ ]:
RE.next_iteration!(s)

p = plot(layout=(2, length(swin_list)), size=(1000, 650))

for (i,swin) in enumerate(values(swin_list))

    y = RE.get_measured(s, swin);
    y2 = RE.get_modeled(s, swin);
    eps = RE.get_noise(s, swin);

    ww = RE.get_wavenumber(s, swin);
    
    plot!(ww, y, label="EM27/SUN", linewidth=2.0) #, ylims=(0, 9))
    plot!(ww, y2, label="Fit", linewidth=1.0, linestyle=:dash, linecolor="black")

    title!("Iteration #$(RE.get_iteration_count(s))", subplot=1, titlefont=(10))
    ylabel!("Signal", subplot=1)
    xlabel!("Wavenumber [cm\$^{-1}\$]", subplot=2, font=(10, "FreeSerif"))
    
    rrms = sqrt(mean((100.0 * (y2 .- y) ./ y).^2))
    
    plot!(ww, (y2 .- y) ./ eps, label="Residual", subplot=2)
    title!(@sprintf("RRMS: %.4f %%", rrms), subplot=2, titlefont=(10))
    ylabel!("Fractions of noise", subplot=2)
   
end

xlabel!("Wavenumber [cm\$^{-1}\$]", subplot=2, font=(10, "FreeSerif"))
display(p)

RE.print_state_vector_update(s.state_vector);

## Run a batch of retrievals

In [ ]:
result_dict = Dict{String, Any}()

result_dict["SZA"] = Float64[]
result_dict["Date"] = DateTime[]
result_dict["JDate"] = Float64[]

for gas_name in (x -> x.gas_name).(gas_dict[idx_want])
    result_dict[gas_name] = Float64[]
end

all_fnames = collect(keys((measurements[idx_want]))) |> sort

@showprogress for fname in all_fnames

    meas = measurements[idx_want][fname]
    
    d0 = meas["start_wavenumber"]
    d1 = meas["delta_wavenumber"]
    
    s = process_spectrum(
        Dict(dispersions[idx_want] => meas["spectrum"]), # Meausred spectrum as dictonary: dispersion => spectrum
        meas["datetime"], # Datetime of measurement
        90.0 - meas["elevation_angle"], # Solar zenith angle
        meas["pressure"], # Measured surface pressure (with physical unit!)
        [d0 - d1, d1, 0.0], # Dispersion coefficients (initial)
        Dict(d.spectral_window => d for d in values(dispersions)),
        meas["noise_std"],
        map_per_file[fname],
        sv_dict[idx_want],
        Dict(dispersions[i] => isrf_dict[i] for i in keys(swin_list)),
        buf;
        max_iter=20
    );

    xgas = RE.calculate_xgas(buf.scene.atmosphere)

    # Add SZA into results
    push!(result_dict["SZA"], buf.scene.solar_zenith)
    # Add datetimes into results
    push!(result_dict["Date"], meas["datetime"])
    push!(result_dict["JDate"], datetime2julian(meas["datetime"]))
    
    # Add XGAS into results
    for (gas_name, gas_value) in xgas
        push!(result_dict[gas_name], gas_value)
    end

end

In [ ]:
# Load the results of the PROFFAST retrievals
df_ref = CSV.File("./example_data/comb_invparms_GSFC_SN250_250722-250722.csv") |> DataFrame;
# Subselect to the measurements we also have:
df_specnames = df_ref.var" spectrum" .|> lstrip
spec_names = (x -> split(x, "/")[end] |> String).(all_fnames)
spec_names = (x -> replace(x, "SM" => "SN")).(spec_names) # Have to replace SM -> SN for this
keep_idx = searchsortedfirst.(Ref(df_specnames), spec_names);

df_ref = df_ref[keep_idx, :];
# Make DateTimes from julian date
df_ref[!, "Date"] .= julian2datetime.(df_ref.var" JulianDate");

In [ ]:
if "CO2" in keys(result_dict)
    scatter(result_dict["Date"], result_dict["CO2"] * 1e6, markershape=:circle, label="GSFC", size=(1000, 300))
    plot!(df_ref.Date, df_ref.var" XCO2_STR", markershape=:square, label="PROFFAST XCO2_STR")
    scatter!(df_ref.Date, df_ref.var" XCO2", markershape=:square, label="PROFFAST XCO2")
    plot!(twinx(), result_dict["Date"], 1 ./ cosd.(result_dict["SZA"]), yaxis = "Airmass",
        color=:black, label=nothing, linewidth=2)
    title!("XCO2")
end

In [ ]:
if "H2O" in keys(result_dict)
    scatter(result_dict["Date"], result_dict["H2O"] * 1e6, markershape=:circle, label="GSFC", size=(900, 300))
    scatter!(df_ref.Date, df_ref.var" XH2O", markmarkershapeer=:square, label="PROFFAST")
    plot!(twinx(), result_dict["Date"], 1 ./ cosd.(result_dict["SZA"]), yaxis = "Airmass",
        color=:black, label="Airmass", linewidth=2)
    title!("XH2O")
end

In [ ]:
if "CH4" in keys(result_dict)
    scatter(result_dict["Date"], result_dict["CH4"] * 1e9, markershape=:circle, label="GSFC", size=(1000, 300))
    scatter!(df_ref.Date, df_ref.var" XCH4" * 1e3, markershape=:square, label="PROFFAST")
    scatter!(df_ref.Date, df_ref.var" XCH4_S5P" * 1e3, markershape=:square, label="PROFFAST")
    plot!(twinx(), result_dict["Date"], 1 ./ cosd.(result_dict["SZA"]), yaxis = "Airmass",
        color=:black, label="Airmass", linewidth=2)
    title!("XCH4")
end

In [ ]:
if "CO" in keys(result_dict)
    scatter(result_dict["Date"], result_dict["CO"] * 1e9, markershape=:circle, label="GSFC", size=(1000, 300))
    scatter!(df_ref.Date, df_ref.var" XCO" * 1e3, markershape=:square, label="PROFFAST")
    plot!(twinx(), result_dict["Date"], 1 ./ cosd.(result_dict["SZA"]), yaxis = "Airmass",
        color=:black, label="Airmass", linewidth=2)
    title!("XCO")
end

In [ ]:
if "O2" in keys(result_dict)
    plot(result_dict["Date"], result_dict["O2"], markershape=:circle, label="GSFC", size=(1000, 300))
    plot!(twinx(), result_dict["Date"], 1 ./ cosd.(result_dict["SZA"]), yaxis = "Airmass",
        color=:green, label=nothing, linewidth=2)
    title!("XO2")
end

In [ ]:
for sv in s.state_vector.state_vector_elements
    p = plot()
    plot!(p, RE.get_wavenumber(s), RE.get_jacobian(s, sv), label="$(sv)")
    ylabel!("∂I/∂SVE");
    display(p)
end


In [ ]:
gases = buf.rt[swin_list[idx_want]].optical_properties.gas_tau
p = plot(size=(1300, 800))

for (gas, gas_tau) in gases
    plot!(p,
        swin_list[idx_want].ww_grid,
        exp.(-sum(gas_tau, dims=2)[:,1]),
        label=gas.gas_name
        )
end
xlabel!("Wavenumber")
ylabel!("Transmission per gas")
display(p)

In [ ]:
RE.calculate_xgas(buf.scene.atmosphere)

In [ ]:
p = plot(yscale=:log10, size=(400, 600))

for (idx,sve) in RE.StateVectorIterator(sv_dict[idx_want], RE.GasLevelScalingFactorSVE)
    col_AK = RE.calculate_scale_factor_AK(
        buf,
        s,
        sve.gas,
        Dict(swin_list[idx_want] => isrf_dict[idx_want]),
        #G::Union{Nothing, AbstractMatrix}=nothing
        );
    
    plot!(p, col_AK, buf.scene.atmosphere.pressure_levels ./ 100, label=sve.gas.gas_name, marker=:circle);

end
yflip!()
xlabel!("AK")
ylabel!("Pressure [hPa]")
display(p)